In [19]:
import os

import time

import re

import pandas as pd

from selenium import webdriver

from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.service import Service

from selenium.webdriver.chrome.options import Options

from selenium.common.exceptions import NoSuchElementException, WebDriverException



# --- 獲取當前時間，用於判斷是否跳過未來月份 ---

# (使用實際執行時的時間)

current_real_time = time.localtime()

current_year = current_real_time.tm_year

current_month = current_real_time.tm_mon

print(f"程式執行時間: {time.strftime('%Y-%m-%d %H:%M:%S')}")

print(f"當前實際年份: {current_year}, 月份: {current_month}")



# ---------- 設定 ----------

# 在目前執行程式的目錄下建立一個子目錄來存放下載的原始CSV檔

DOWNLOAD_DIR = os.path.join(os.getcwd(), "mops_csv_raw")

os.makedirs(DOWNLOAD_DIR, exist_ok=True)

print(f"設定下載目錄: {DOWNLOAD_DIR}")



chrome_options = Options()

# 設定 Chrome 瀏覽器偏好，將檔案自動下載到指定目錄

chrome_options.add_experimental_option("prefs", {

    "download.default_directory": DOWNLOAD_DIR,

    "download.prompt_for_download": False, # <--- 重要：設為 False，不跳出詢問視窗，直接下載

    "download.directory_upgrade": True,     # 允許下載目錄變更

    "plugins.always_open_pdf_externally": True, # 如果有PDF，也直接下載

    "safeBrowse.enabled": True             # 開啟安全瀏覽 (可選)

})

# 使用無頭模式執行 Chrome (背景執行，不顯示瀏覽器視窗)

chrome_options.add_argument("--headless=new") # 使用最新的 headless 模式

chrome_options.add_argument("--disable-gpu") # 無頭模式下通常需要

chrome_options.add_argument("--window-size=1920,1080") # 指定視窗大小有時有幫助

chrome_options.add_argument("--no-sandbox") # 在某些環境 (如 Docker) 可能需要

chrome_options.add_argument("--disable-dev-shm-usage") # 解決資源限制問題



# 確保 chromedriver 已安裝且在系統路徑中，或提供其路徑

try:

    # 假設 chromedriver 在系統 PATH 中

    # 如果不在 PATH，可以用 service = Service(executable_path='/path/to/chromedriver')

    service = Service()

    driver = webdriver.Chrome(service=service, options=chrome_options)

    print("WebDriver 啟動成功 (Headless Mode)")

except WebDriverException as e:

    print(f"WebDriver 啟動失敗: {e}")

    print("請確保 chromedriver 已安裝並在系統 PATH 中，或者在 Service() 中提供 chromedriver 的絕對路徑。")

    exit() # 如果驅動程式啟動失敗則退出



# 定義要抓取的年份和市場類別

# 注意：根據目前日期 (2025-04-12)，2025年的資料可能尚未完整。

# 如果您想嘗試包含 2025 年的資料，請調整 range(2018, 2026)。

# 目前設定 range(2018, 2025) 會抓取 2018, 2019, 2020, 2021, 2022, 2023, 2024 年。

# 若要包含 2025 年前幾個月的資料，請改為 range(2018, current_year + 1) 或 range(2018, 2026)

years_to_fetch = range(2017, current_year + 1) # 抓取從 2017 到 當前年份

markets = [

    ("sii", "國內上市", 0), # market_code, market_name, type_flag (0/1 的意義依 MOPS 網站定義)

    ("sii", "國外上市", 1),

    ("otc", "國內上櫃", 0),

    ("otc", "國外上櫃", 1),

]



# ---------- 下載 CSV ----------

print("\n--- 開始下載 CSV 檔案 ---")

for year in years_to_fetch:

    roc_year = year - 1911 # 西元年轉換為民國年

    print(f"\n處理年份: {year} (民國 {roc_year} 年)")

    start_month = 1

    end_month = 12

    if year == 2017:

        start_month = 3

    if year == current_year:

        end_month = current_month

        if current_month < 3: # 如果當前月份小於 3 月，則不抓取當年的資料，因為使用者要求到 2017/3

            continue



    for month in range(start_month, end_month + 1):

        # 檢查是否需要跳過未來的月份或當月 (因為資料可能尚未公佈)

        # current_year 和 current_month 是在程式開始時獲取的實際時間

        if year > current_year or (year == current_year and month > current_month): # 修改為 > 而不是 >=，因為我們已經設定了 end_month

            # 如果年份大於當前年份，或年份相同但月份大於當前月份

            # 通常需等到下個月中旬後才會公佈上個月的完整資料

            print(f"  [跳過] 未來月份資料: {year}-{month:02d}")

            continue # 跳過這個月份，處理下一個



        for market_code, market_name, type_flag in markets:

            # 根據 MOPS 慣例構造檔名

            base_file_name = f"t21sc03_{roc_year}_{month}"

            file_name_with_flag = f"{base_file_name}_{type_flag}.csv"

            file_name_without_flag = f"{base_file_name}.csv"

            file_path_with_flag = os.path.join(DOWNLOAD_DIR, file_name_with_flag)

            file_path_without_flag = os.path.join(DOWNLOAD_DIR, file_name_without_flag)



            # 檢查檔案是否已存在 (任一種格式)

            if os.path.exists(file_path_with_flag) or os.path.exists(file_path_without_flag):

                print(f"  [跳過] 已存在 {file_name_with_flag} 或 {file_name_without_flag}")

                continue



            # 構造特定報表的網頁 URL

            url_with_flag = f"https://mopsov.twse.com.tw/nas/t21/{market_code}/t21sc03_{roc_year}_{month}_{type_flag}.html"

            url_without_flag = f"https://mopsov.twse.com.tw/nas/t21/{market_code}/t21sc03_{roc_year}_{month}.html"



            # For the year 2017, try downloading without the type flag first

            if year == 2017:

                print(f"  嘗試下載 (no flag): {file_name_without_flag} 從 {url_without_flag}")

                try:

                    driver.get(url_without_flag)

                    time.sleep(5)

                    try:

                        csv_btn = driver.find_element(By.CSS_SELECTOR, "input[type='button'][name='download'][value='另存CSV']")

                        csv_btn.click()

                        print(f"    點擊 '另存CSV' 按鈕 (自動觸發下載)...")

                        time.sleep(7)

                        # --- Download check logic (same as before) ---

                        download_complete = False

                        max_checks = 5

                        check_interval = 1

                        print(f"    檢查檔案是否下載完成 (最多等待 {max_checks * check_interval:.1f} 秒)...")

                        for i in range(max_checks):

                            if os.path.exists(file_path_without_flag):

                                download_complete = True

                                time.sleep(0.5)

                                print(f"    檔案出現: {file_name_without_flag}")

                                break

                            temp_file_path = file_path_without_flag + ".crdownload"

                            if os.path.exists(temp_file_path):

                                print(f"    [等待中 {i+1}/{max_checks}] 偵測到暫存檔 {os.path.basename(temp_file_path)}...")

                            else:

                                print(f"    [等待中 {i+1}/{max_checks}] 檔案尚未出現...")

                            time.sleep(check_interval)



                        if download_complete:

                            print(f"  ✅ 下載完成 {file_name_without_flag}")

                            continue # Skip to the next iteration of the outer loop

                        else:

                            temp_file_path = file_path_without_flag + ".crdownload"

                            if os.path.exists(file_path_without_flag):

                                print(f"  ✅ 下載完成 {file_name_without_flag} (最後檢查發現)")

                                continue

                            elif os.path.exists(temp_file_path):

                                print(f"  ❌ 下載失敗 {file_name_without_flag} (下載未完成或卡住)")

                            else:

                                print(f"  ❌ 下載失敗 {file_name_without_flag} (檢查超時)")

                    except NoSuchElementException:

                        print(f"    [無按鈕] 在 {url_without_flag} 找不到下載按鈕 (可能無資料)")

                except WebDriverException as e:

                    print(f"  [錯誤] WebDriver 錯誤於 {file_name_without_flag}: {e}")

                    time.sleep(5)

                except Exception as e:

                    print(f"  [錯誤] 未預期錯誤於 {file_name_without_flag}: {e}")



            # Try the original logic with the type flag

            print(f"  嘗試下載 (with flag): {file_name_with_flag} 從 {url_with_flag}")

            try:

                driver.get(url_with_flag)

                time.sleep(5)

                try:

                    csv_btn = driver.find_element(By.CSS_SELECTOR, "input[type='button'][name='download'][value='另存CSV']")

                    csv_btn.click()

                    print(f"    點擊 '另存CSV' 按鈕 (自動觸發下載)...")

                    time.sleep(7)

                    # --- Download check logic (same as before) ---

                    download_complete = False

                    max_checks = 5

                    check_interval = 1

                    print(f"    檢查檔案是否下載完成 (最多等待 {max_checks * check_interval:.1f} 秒)...")

                    for i in range(max_checks):

                        if os.path.exists(file_path_with_flag):

                            download_complete = True

                            time.sleep(0.5)

                            print(f"    檔案出現: {file_name_with_flag}")

                            break

                        temp_file_path = file_path_with_flag + ".crdownload"

                        if os.path.exists(temp_file_path):

                            print(f"    [等待中 {i+1}/{max_checks}] 偵測到暫存檔 {os.path.basename(temp_file_path)}...")

                        else:

                            print(f"    [等待中 {i+1}/{max_checks}] 檔案尚未出現...")

                        time.sleep(check_interval)



                    if download_complete:

                        print(f"  ✅ 下載完成 {file_name_with_flag}")

                    else:

                        temp_file_path = file_path_with_flag + ".crdownload"

                        if os.path.exists(file_path_with_flag):

                            print(f"  ✅ 下載完成 {file_name_with_flag} (最後檢查發現)")

                        elif os.path.exists(temp_file_path):

                            print(f"  ❌ 下載失敗 {file_name_with_flag} (下載未完成或卡住)")

                        else:

                            print(f"  ❌ 下載失敗 {file_name_with_flag} (檢查超時)")

                except NoSuchElementException:

                    print(f"    [無按鈕] 在 {url_with_flag} 找不到下載按鈕 (可能無資料)")

            except WebDriverException as e:

                print(f"  [錯誤] WebDriver 錯誤於 {file_name_with_flag}: {e}")

                time.sleep(5)

            except Exception as e:

                print(f"  [錯誤] 未預期錯誤於 {file_name_with_flag}: {e}")



# ---------- 整合資料 ----------

print("\n--- 開始整合資料 ---")



# 定義最終輸出檔案中想要保留的欄位名稱

# (請確保這些名稱與 MOPS CSV 檔案中的實際欄位名稱一致或進行映射)

columns_to_keep = [

    "年度", "月份", "市場別", "公司代號", "公司名稱", "產業別",

    "營業收入-當月營收", "營業收入-上月營收", "營業收入-去年當月營收",

    "營業收入-上月比較增減(%)", "營業收入-去年同月增減(%)",

    "累計營業收入-當月累計營收", "累計營業收入-去年累計營收",

    "累計營業收入-前期比較增減(%)",

    "備註" # 保留備註欄位，可能包含重要資訊

]



# 將內部代碼映射回易於閱讀的市場名稱

market_map = {

    "sii_0": "國內上市", "sii_1": "國外上市",

    "otc_0": "國內上櫃", "otc_1": "國外上櫃"

}



all_data = [] # 用於存放從每個 CSV 讀取的 DataFrame



# 遍歷下載目錄中的所有檔案

try:

    downloaded_files = os.listdir(DOWNLOAD_DIR)

    print(f"在 {DOWNLOAD_DIR} 中找到 {len(downloaded_files)} 個檔案，開始處理...")

except FileNotFoundError:

    print(f"錯誤：找不到下載目錄 {DOWNLOAD_DIR}。請確保下載步驟成功執行。")

    downloaded_files = [] # 如果目錄不存在，則檔案列表為空



for file in downloaded_files:

    # 使用正規表達式解析檔名，提取年份、月份、類型標誌

    # re.IGNORECASE 讓大小寫不敏感，增加一點彈性

    match = re.match(r"t21sc03_(\d+)_(\d{1,2})(_(\d))?\.csv", file, re.IGNORECASE)

    if not match:

        # 跳過不符合預期格式的檔案

        if file.endswith('.csv'): # 只對 CSV 檔顯示警告

            print(f"  [跳過] 檔名格式不符: {file}")

        continue # 處理下一個檔案



    roc_year_str, month_str, _, type_flag_str = match.groups()

    try:

        roc_year = int(roc_year_str)

        month = int(month_str)

        year = roc_year + 1911 # 民國年轉回西元年

        type_flag = int(type_flag_str) if type_flag_str else None

    except ValueError:

        print(f"  [跳過] 無法從檔名解析數字: {file}")

        continue



    # 根據檔名中的類型標誌推斷市場名稱

    # 這假設檔名中的 sii/otc 和 type_flag 是可靠的

    # 更可靠的方式是在下載時就記錄好對應關係，但目前依賴檔名

    market_key_part = "sii" if "sii" in file.lower() else "otc"

    market_key = f"{market_key_part}_{type_flag}" if type_flag is not None else f"{market_key_part}_None" # Handle case without type flag

    market_name = market_map.get(f"{market_key_part}_{type_flag if type_flag is not None else 0}", f"未知市場({market_key})") # Default to 0 if no flag



    file_path = os.path.join(DOWNLOAD_DIR, file)

    print(f"  處理檔案: {file} (Year: {year}, Month: {month}, Market: {market_name}, Type Flag: {type_flag})")



    try:

        # 使用 'big5' 編碼讀取 CSV，這對 MOPS 資料至關重要

        # 初始將所有欄位讀取為字串 (dtype=str)，以避免類型錯誤，特別是公司代號

        # na_filter=False 保留空字串，而不是將其視為 NaN (Not a Number)

        df = pd.read_csv(file_path, encoding="big5", dtype=str, na_filter=False, thousands=',')



        # --- 基本資料驗證與清理 ---

        # 檢查必要的欄位是否存在 (使用 MOPS CSV 中的確切名稱)

        required_cols = ['公司代號', '公司名稱']

        if not all(col in df.columns for col in required_cols):

            # MOPS 檔案有時可能只包含標頭/註腳文字，或格式錯誤

            # 簡單檢查：如果行數很少，或 '公司代號' 欄位不包含看起來像數字的內容

            if df.shape[0] < 5 or not df['公司代號'].astype(str).str.contains(r'^\d+$', na=False).any():

                print(f"    [警告] 檔案可能為空或僅包含標頭/註腳: {file}")

                continue # 跳過此檔案

            else:

                missing = [col for col in required_cols if col not in df.columns]

                print(f"    [警告] 缺少欄位: {', '.join(missing)}，但仍嘗試處理: {file}")

                # 根據需求決定是否要基於此警告跳過或繼續



        # 移除 '合計' 行 (如果存在)

        # 使用 .copy() 避免 SettingWithCopyWarning

        if '公司代號' in df.columns:

            df = df[df['公司代號'].astype(str).str.strip() != '合計'].copy()



        # 再次清理：只保留 '公司代號' 看起來是純數字的行

        if '公司代號' in df.columns:

            # .str.match(r'^\d+$') 確保是純數字代碼

            df = df[df['公司代號'].astype(str).str.strip().str.match(r'^\d+$')].copy()



        if df.empty:

            print(f"    [資訊] 過濾後無有效資料: {file}")

            continue # 如果清理後沒有有效資料行，則跳過



        # --- 添加元數據欄位 ---

        # 在 DataFrame 開頭插入年份、月份、市場別

        df.insert(0, "年度", str(year))

        df.insert(1, "月份", str(month).zfill(2)) # 月份補零 (例如 '01', '02')

        df.insert(2, "市場別", market_name)



        # --- 欄位選擇 ---

        # 只保留我們需要的欄位

        # 確保只選擇 DataFrame 中實際存在的欄位，避免錯誤

        existing_cols_to_keep = [col for col in columns_to_keep if col in df.columns]

        missing_cols = [col for col in columns_to_keep if col not in df.columns]

        if missing_cols:

            # 如果 columns_to_keep 中定義的某些欄位在該 CSV 不存在，則提示

            print(f"    [注意] 檔案 {file} 缺少預期欄位: {', '.join(missing_cols)}")

            # 可以選擇在這裡為缺失的欄位添加空值，或保持現狀



        df_filtered = df[existing_cols_to_keep].copy() # 只取需要的欄位



        # 將處理好的 DataFrame 加入列表

        all_data.append(df_filtered)

        print(f"    ✔ 讀取並處理成功: {file}")



    except FileNotFoundError:

        print(f"  [錯誤] 檔案未找到 (可能在下載步驟失敗): {file}")

    except pd.errors.EmptyDataError:

        print(f"  [錯誤] 檔案是空的，無法讀取: {file}")

    except UnicodeDecodeError:

        print(f"  [錯誤] Big5 解碼失敗，請檢查檔案內容或嘗試 UTF-8: {file}")

        # 可以選擇在這裡加入 try-except 嘗試用 UTF-8 讀取作為備用方案

        # try:

        #     df = pd.read_csv(file_path, encoding="utf-8", dtype=str, na_filter=False)

        #     # ... 重複驗證和處理步驟 ...

        # except Exception as fallback_e:

        #     print(f"      [錯誤] 使用 UTF-8 讀取也失敗: {file} - {fallback_e}")

    except Exception as e:

        # 捕捉讀取或處理過程中的其他潛在錯誤

        print(f"  [錯誤] 讀取或處理失敗: {file} - {e}")

        continue # 處理下一個檔案



# ---------- 儲存最終結果 ----------

# 決定最終輸出的檔名

output_filename = f"mops_monthly_revenue_{years_to_fetch.start}-{years_to_fetch.stop - 1}_Mar2017-Mar{current_year}.csv" # Updated filename



if all_data:

    print("\n--- 資料整合中 ---")

    # 將列表中所有的 DataFrame 合併成一個大的 DataFrame

    # ignore_index=True 會重新生成索引

    final_df = pd.concat(all_data, ignore_index=True)

    print(f"總共整合 {len(final_df)} 筆資料")



    # 確保最終 DataFrame 的欄位順序與 columns_to_keep 一致

    # reindex 會添加那些在某些檔案中可能缺失的欄位 (值為 NaN)

    final_df = final_df.reindex(columns=columns_to_keep)



    # 可選：將數值欄位從字串轉換回數值類型

    # 要小心處理可能的錯誤 ('coerce' 會將無法轉換的值變成 NaN)

    numeric_cols = [

        "營業收入-當月營收", "營業收入-上月營收", "營業收入-去年當月營收",

        "營業收入-上月比較增減(%)", "營業收入-去年同月增減(%)",

        "累計營業收入-當月累計營收", "累計營業收入-去年累計營收",

        "累計營業收入-前期比較增減(%)"

    ]

    print("轉換數值欄位 (若有)...")

    for col in numeric_cols:

        if col in final_df.columns:

            # 移除千分位逗號 (如果有的話，雖然 read_csv 加了 thousands=',')

            final_df[col] = final_df[col].astype(str).str.replace(',', '', regex=False)

            # 轉換為數值，無法轉換的設為 NaN

            final_df[col] = pd.to_numeric(final_df[col], errors='coerce')

    print("數值欄位轉換完成 (無法轉換的值已設為 NaN)")



    # 將最終的 DataFrame 儲存為 CSV 檔案

    # 使用 utf-8-sig 編碼，以便 Excel 正確開啟包含中文的檔案

    try:

        final_df.to_csv(output_filename, index=False, encoding="utf-8-sig")

        print(f"\n🎉 資料整合完成並儲存至: {output_filename}")

    except Exception as e:

        print(f"\n❌ 儲存最終 CSV 失敗: {e}")

else:

    # 如果 all_data 列表是空的 (沒有任何檔案成功處理)

    print("\n❌ 沒有資料成功讀取或整合，無法產生最終檔案。")



print("\n--- 程式執行完畢 ---")

SyntaxError: invalid non-printable character U+00A0 (2187670923.py, line 55)

In [21]:
import os
import time
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, WebDriverException

# 取得當前年月
current_time = time.localtime()
current_year = current_time.tm_year
current_month = current_time.tm_mon

# 設定下載目錄
DOWNLOAD_DIR = os.path.join(os.getcwd(), "mops_csv_raw")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# 設定 Chrome options
chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "directory_upgrade": True,
    "safebrowsing.enabled": True
})
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 啟動 Chrome Driver
try:
    service = Service()
    driver = webdriver.Chrome(service=service, options=chrome_options)
    print("✅ WebDriver 啟動成功")
except WebDriverException as e:
    print(f"❌ WebDriver 啟動失敗: {e}")
    exit()

# 設定年份與市場別
years_to_fetch = range(2017, current_year + 1)
markets = [
    ("sii", "國內上市", 0),
    ("sii", "國外上市", 1),
    ("otc", "國內上櫃", 0),
    ("otc", "國外上櫃", 1),
]

# 開始下載
print("\n📥 開始下載 CSV...")
for year in years_to_fetch:
    roc_year = year - 1911
    for month in range(1, 13):
        if year > current_year or (year == current_year and month >= current_month):
            continue
        for market_code, market_name, type_flag in markets:
            file_name = f"t21sc03_{roc_year}_{month}_{type_flag}.csv"
            file_path = os.path.join(DOWNLOAD_DIR, file_name)
            if os.path.exists(file_path):
                print(f"✅ 已存在 {file_name}，跳過")
                continue
            url = f"https://mopsov.twse.com.tw/nas/t21/{market_code}/t21sc03_{roc_year}_{month}_{type_flag}.html"
            print(f"🔄 下載中：{file_name}")
            try:
                driver.get(url)
                time.sleep(2.5)
                try:
                    csv_btn = driver.find_element(By.CSS_SELECTOR, "input[type='button'][name='download'][value='另存CSV']")
                    csv_btn.click()
                    time.sleep(1.5)  # 不等待完成，直接下一筆
                    print(f"   ⏭️ 已點擊另存CSV")
                except NoSuchElementException:
                    print(f"   ⚠️ 無下載按鈕（可能無資料）")
                    continue
            except Exception as e:
                print(f"   ❌ 錯誤：{e}")
                continue

driver.quit()
print("\n✅ 所有下載已啟動，開始整合資料")

# 整合資料
columns_to_keep = [
    "年度", "月份", "市場別", "公司代號", "公司名稱", "產業別",
    "營業收入-當月營收", "營業收入-上月營收", "營業收入-去年當月營收",
    "營業收入-上月比較增減(%)", "營業收入-去年同月增減(%)",
    "累計營業收入-當月累計營收", "累計營業收入-去年累計營收",
    "累計營業收入-前期比較增減(%)", "備註"
]

market_map = {
    "sii_0": "國內上市", "sii_1": "國外上市",
    "otc_0": "國內上櫃", "otc_1": "國外上櫃"
}

all_data = []

for file in os.listdir(DOWNLOAD_DIR):
    match = re.match(r"t21sc03_(\d+)_(\d{1,2})_(\d)\.csv", file)
    if not match:
        continue
    roc_year, month, type_flag = match.groups()
    year = int(roc_year) + 1911
    market_key = "sii" if "sii" in file.lower() else "otc"
    market_name = market_map.get(f"{market_key}_{type_flag}", "未知市場")
    file_path = os.path.join(DOWNLOAD_DIR, file)
    try:
        df = pd.read_csv(file_path, encoding="big5", dtype=str, na_filter=False, thousands=',')
        if '公司代號' not in df.columns:
            continue
        df = df[df['公司代號'].astype(str).str.strip().str.match(r'^\d+$')]
        if df.empty:
            continue
        df.insert(0, "年度", str(year))
        df.insert(1, "月份", str(month).zfill(2))
        df.insert(2, "市場別", market_name)
        existing_cols = [col for col in columns_to_keep if col in df.columns]
        df_filtered = df[existing_cols].copy()
        all_data.append(df_filtered)
        print(f"📄 已整合：{file}")
    except Exception as e:
        print(f"⚠️ 錯誤處理 {file}：{e}")
        continue

# 儲存成一個 CSV
output_filename = f"mops_monthly_revenue_{years_to_fetch.start}-{years_to_fetch.stop - 1}.csv"

if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    final_df = final_df.reindex(columns=columns_to_keep)
    numeric_cols = [
        "營業收入-當月營收", "營業收入-上月營收", "營業收入-去年當月營收",
        "營業收入-上月比較增減(%)", "營業收入-去年同月增減(%)",
        "累計營業收入-當月累計營收", "累計營業收入-去年累計營收",
        "累計營業收入-前期比較增減(%)"
    ]
    for col in numeric_cols:
        if col in final_df.columns:
            final_df[col] = final_df[col].astype(str).str.replace(',', '', regex=False)
            final_df[col] = pd.to_numeric(final_df[col], errors='coerce')
    final_df.to_csv(output_filename, index=False, encoding="utf-8-sig")
    print(f"\n✅ 資料整合完成，已儲存：{output_filename}")
else:
    print("\n⚠️ 沒有成功整合任何資料")


✅ WebDriver 啟動成功

📥 開始下載 CSV...
🔄 下載中：t21sc03_106_1_0.csv
   ⏭️ 已點擊另存CSV
🔄 下載中：t21sc03_106_1_1.csv
   ⏭️ 已點擊另存CSV
🔄 下載中：t21sc03_106_1_0.csv
   ⏭️ 已點擊另存CSV
🔄 下載中：t21sc03_106_1_1.csv
   ⏭️ 已點擊另存CSV
🔄 下載中：t21sc03_106_2_0.csv
   ⏭️ 已點擊另存CSV
🔄 下載中：t21sc03_106_2_1.csv
   ⏭️ 已點擊另存CSV
🔄 下載中：t21sc03_106_2_0.csv
   ⏭️ 已點擊另存CSV
🔄 下載中：t21sc03_106_2_1.csv
   ⏭️ 已點擊另存CSV
🔄 下載中：t21sc03_106_3_0.csv
   ⏭️ 已點擊另存CSV
🔄 下載中：t21sc03_106_3_1.csv
   ⏭️ 已點擊另存CSV
🔄 下載中：t21sc03_106_3_0.csv
   ⏭️ 已點擊另存CSV
🔄 下載中：t21sc03_106_3_1.csv
   ⏭️ 已點擊另存CSV
🔄 下載中：t21sc03_106_4_0.csv


KeyboardInterrupt: 

In [26]:
import os
import time
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

# === 時間設定 ===
current_time = time.localtime()
current_year = current_time.tm_year
current_month = current_time.tm_mon

# === 儲存與編碼設定 ===
DOWNLOAD_DIR = os.path.join(os.getcwd(), "mops_csv_raw")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "directory_upgrade": True,
    "safebrowsing.enabled": True
})
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")

driver = webdriver.Chrome(service=Service(), options=chrome_options)

# === 年度與市場別設定 ===
years_to_fetch = range(2017, current_year + 1)
markets = [
    ("sii", "國內上市", 0),
    ("sii", "國外上市", 1),
    ("otc", "國內上櫃", 0),
    ("otc", "國外上櫃", 1),
]

market_map = {
    "sii_0": "國內上市", "sii_1": "國內上櫃",
    "otc_0": "國內上櫃", "otc_1": "國外上櫃"
}

columns_to_keep = [
    "年度", "月份", "市場別", "公司代號", "公司名稱", "產業別",
    "營業收入-當月營收", "營業收入-上月營收", "營業收入-去年當月營收",
    "營業收入-上月比較增減(%)", "營業收入-去年同月增減(%)",
    "累計營業收入-當月累計營收", "累計營業收入-去年累計營收",
    "累計營業收入-前期比較增減(%)", "備註"
]

print("\n📥 開始下載與整合每月資料...")
for year in years_to_fetch:
    roc_year = year - 1911
    for month in range(1, 13):
        if year > current_year or (year == current_year and month >= current_month):
            continue

        print(f"\n📅 處理：{year}-{month:02d}")
        pre_download_files = set(os.listdir(DOWNLOAD_DIR))  # 先記錄現有檔案
        downloaded_this_month = []

        # === 下載當月四市場資料 ===
        for market_code, market_name, type_flag in markets:
            url = f"https://mopsov.twse.com.tw/nas/t21/{market_code}/t21sc03_{roc_year}_{month}_{type_flag}.html"
            try:
                driver.get(url)
                time.sleep(2)
                try:
                    btn = driver.find_element(By.CSS_SELECTOR, "input[type='button'][name='download'][value='另存CSV']")
                    btn.click()
                    time.sleep(1.5)
                    print(f"✅ 已點擊下載：{market_map[f'{market_code}_{type_flag}']}")
                except NoSuchElementException:
                    print(f"⚠️ 無下載按鈕（{market_map[f'{market_code}_{type_flag}']}）")
                    continue
            except Exception as e:
                print(f"❌ 錯誤：{e}")
                continue

        # === 等待新檔案出現 ===
        time.sleep(2)
        post_download_files = set(os.listdir(DOWNLOAD_DIR))
        new_files = list(post_download_files - pre_download_files)
        new_files = [f for f in new_files if f.endswith(".csv") and f"_{roc_year}_{month}_" in f or f.startswith(f"t21sc03_{roc_year}_{month}")]

        if not new_files:
            print("⚠️ 本月無新下載資料")
            continue

        # === 整合單月資料 ===
        combined_monthly_data = []
        for file in new_files:
            file_path = os.path.join(DOWNLOAD_DIR, file)
            try:
                df = pd.read_csv(file_path, encoding="big5", dtype=str, na_filter=False)
                if '公司代號' not in df.columns:
                    continue
                df = df[df['公司代號'].astype(str).str.strip().str.match(r'^\d+$')]
                market_match = re.search(r"_(sii|otc)", file)
                type_match = re.search(r"_(\d)\.csv", file)
                if not market_match or not type_match:
                    continue
                market_code = market_match.group(1)
                type_flag = type_match.group(1)
                market_key = f"{market_code}_{type_flag}"
                market_name = market_map.get(market_key, "未知市場")
                df.insert(0, "年度", str(year))
                df.insert(1, "月份", str(month).zfill(2))
                df.insert(2, "市場別", market_name)
                df = df[[col for col in columns_to_keep if col in df.columns]]
                combined_monthly_data.append(df)
                print(f"📄 合併檔案：{file}")
            except Exception as e:
                print(f"❌ 錯誤讀取 {file}：{e}")

        if combined_monthly_data:
            final_df = pd.concat(combined_monthly_data, ignore_index=True)
            final_df = final_df.reindex(columns=columns_to_keep)
            output_file = f"mops_{year}_{str(month).zfill(2)}.csv"
            final_df.to_csv(output_file, index=False, encoding="utf-8-sig")
            print(f"📦 完成單月整合：{output_file}")
        else:
            print("⚠️ 本月沒有有效資料可以整合")

driver.quit()
print("\n✅ 全部月份處理完畢")



📥 開始下載與整合每月資料...

📅 處理：2017-01
✅ 已點擊下載：國內上市
✅ 已點擊下載：國內上櫃
✅ 已點擊下載：國內上櫃
✅ 已點擊下載：國外上櫃
❌ 錯誤讀取 t21sc03_106_1 (1).csv：'big5' codec can't decode byte 0x87 in position 4: illegal multibyte sequence
❌ 錯誤讀取 t21sc03_106_1.csv：'big5' codec can't decode byte 0x87 in position 4: illegal multibyte sequence
❌ 錯誤讀取 t21sc03_106_1 (2).csv：'big5' codec can't decode byte 0x87 in position 4: illegal multibyte sequence
❌ 錯誤讀取 t21sc03_106_1 (3).csv：'big5' codec can't decode byte 0x87 in position 4: illegal multibyte sequence
⚠️ 本月沒有有效資料可以整合

📅 處理：2017-02
✅ 已點擊下載：國內上市


KeyboardInterrupt: 

In [27]:
import os
import time
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# === 設定基本參數 ===
current_time = time.localtime()
current_year = current_time.tm_year
current_month = current_time.tm_mon
DOWNLOAD_DIR = os.path.join(os.getcwd(), "mops_csv_raw")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# === Selenium 設定 ===
chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "directory_upgrade": True,
    "safebrowsing.enabled": True
})
# 可以註解這行看到畫面：chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(service=Service(), options=chrome_options)

# === 年 & 市場設定 ===
years_to_fetch = range(2017, current_year + 1)
markets = [
    ("sii", "國內上市", 0),
    ("sii", "國外上市", 1),
    ("otc", "國內上櫃", 0),
    ("otc", "國外上櫃", 1),
]
columns_to_keep = [
    "年度", "月份", "市場別", "公司代號", "公司名稱", "產業別",
    "營業收入-當月營收", "營業收入-上月營收", "營業收入-去年當月營收",
    "營業收入-上月比較增減(%)", "營業收入-去年同月增減(%)",
    "累計營業收入-當月累計營收", "累計營業收入-去年累計營收",
    "累計營業收入-前期比較增減(%)", "備註"
]

print("\n📥 開始每月資料處理...")
for year in years_to_fetch:
    roc_year = year - 1911
    for month in range(1, 13):
        if year > current_year or (year == current_year and month >= current_month):
            continue

        print(f"\n📅 處理 {year}-{month:02d}...")
        combined_monthly_data = []

        for market_code, market_name, type_flag in markets:
            url = f"https://mopsov.twse.com.tw/nas/t21/{market_code}/t21sc03_{roc_year}_{month}_{type_flag}.html"
            csv_filename = f"{year}_{str(month).zfill(2)}_{market_name}.csv"
            csv_path = os.path.join(DOWNLOAD_DIR, csv_filename)

            driver.get(url)
            time.sleep(1)

            before_files = set(os.listdir(DOWNLOAD_DIR))

            try:
                # 等最多 5 秒等待按鈕出現
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='button'][name='download'][value='另存CSV']"))
                )
                btn = driver.find_element(By.CSS_SELECTOR, "input[type='button'][name='download'][value='另存CSV']")
                btn.click()
                time.sleep(2)

                after_files = set(os.listdir(DOWNLOAD_DIR))
                new_files = list(after_files - before_files)
                if not new_files:
                    raise Exception("找不到新下載的 CSV")

                latest_file = max(
                    [os.path.join(DOWNLOAD_DIR, f) for f in new_files if f.endswith('.csv')],
                    key=os.path.getctime
                )
                os.rename(latest_file, csv_path)
                print(f"✅ 下載成功：{csv_filename}")

            except Exception as e:
                print(f"⚠️ 無資料或錯誤，建立空檔：{csv_filename}")
                pd.DataFrame(columns=columns_to_keep).to_csv(csv_path, index=False, encoding="utf-8-sig")

            # 嘗試讀檔案
            try:
                df = pd.read_csv(csv_path, encoding="utf-8-sig", dtype=str)
                if df.empty or '公司代號' not in df.columns:
                    raise ValueError("無有效資料")
                df = df[df['公司代號'].astype(str).str.strip().str.match(r'^\d+$')]
                if df.empty:
                    raise ValueError("過濾後無資料")
                df.insert(0, "年度", str(year))
                df.insert(1, "月份", str(month).zfill(2))
                df.insert(2, "市場別", market_name)
                df = df[[col for col in columns_to_keep if col in df.columns]]
                combined_monthly_data.append(df)
            except Exception as e:
                print(f"⚠️ 無有效資料，跳過讀取：{csv_filename}")

        # 整合所有市場資料
        if combined_monthly_data:
            final_df = pd.concat(combined_monthly_data, ignore_index=True)
            final_df = final_df.reindex(columns=columns_to_keep)

            numeric_cols = [
                "營業收入-當月營收", "營業收入-上月營收", "營業收入-去年當月營收",
                "營業收入-上月比較增減(%)", "營業收入-去年同月增減(%)",
                "累計營業收入-當月累計營收", "累計營業收入-去年累計營收",
                "累計營業收入-前期比較增減(%)"
            ]
            for col in numeric_cols:
                if col in final_df.columns:
                    final_df[col] = final_df[col].astype(str).str.replace(',', '', regex=False)
                    final_df[col] = pd.to_numeric(final_df[col], errors='coerce')

            output_file = f"mops_{year}_{str(month).zfill(2)}.csv"
            final_df.to_csv(output_file, index=False, encoding="utf-8-sig")
            print(f"📦 整合完成：{output_file}")
        else:
            print(f"❌ 沒有資料可整合：{year}-{month:02d}")

driver.quit()
print("\n✅ 所有月份下載與整合作業完成！")



📥 開始每月資料處理...

📅 處理 2017-01...
✅ 下載成功：2017_01_國內上市.csv
✅ 下載成功：2017_01_國外上市.csv
✅ 下載成功：2017_01_國內上櫃.csv
✅ 下載成功：2017_01_國外上櫃.csv
📦 整合完成：mops_2017_01.csv

📅 處理 2017-02...
✅ 下載成功：2017_02_國內上市.csv
✅ 下載成功：2017_02_國外上市.csv
✅ 下載成功：2017_02_國內上櫃.csv
✅ 下載成功：2017_02_國外上櫃.csv
📦 整合完成：mops_2017_02.csv

📅 處理 2017-03...
✅ 下載成功：2017_03_國內上市.csv
✅ 下載成功：2017_03_國外上市.csv
✅ 下載成功：2017_03_國內上櫃.csv
✅ 下載成功：2017_03_國外上櫃.csv
📦 整合完成：mops_2017_03.csv

📅 處理 2017-04...
✅ 下載成功：2017_04_國內上市.csv
✅ 下載成功：2017_04_國外上市.csv
✅ 下載成功：2017_04_國內上櫃.csv
✅ 下載成功：2017_04_國外上櫃.csv
📦 整合完成：mops_2017_04.csv

📅 處理 2017-05...
✅ 下載成功：2017_05_國內上市.csv
✅ 下載成功：2017_05_國外上市.csv
✅ 下載成功：2017_05_國內上櫃.csv
✅ 下載成功：2017_05_國外上櫃.csv
📦 整合完成：mops_2017_05.csv

📅 處理 2017-06...
✅ 下載成功：2017_06_國內上市.csv
✅ 下載成功：2017_06_國外上市.csv
✅ 下載成功：2017_06_國內上櫃.csv
✅ 下載成功：2017_06_國外上櫃.csv
📦 整合完成：mops_2017_06.csv

📅 處理 2017-07...
✅ 下載成功：2017_07_國內上市.csv
✅ 下載成功：2017_07_國外上市.csv
✅ 下載成功：2017_07_國內上櫃.csv
✅ 下載成功：2017_07_國外上櫃.csv
📦 整合完成：mops_2017_07.csv

📅 處理 2017-08...
✅ 下載成功：20